<a href="https://colab.research.google.com/github/BrajanNieto/KNN-HPC/blob/main/Proyecto_HPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Computación Paralela — k-NN distribuido con MPI (mpi4py)**

---

*   Este notebook aplica conceptos de cómputo distribuido con MPI para acelerar el algoritmo **k-Nearest Neighbors (k-NN)**. El flujo implementado: genera un dataset multicategoría, distribuye el conjunto de entrenamiento entre procesos (Scatter), difunde el conjunto de prueba (Bcast), calcula distancias localmente por proceso, reúne resultados parciales (Gather) y aplica voto mayoritario para la predicción final.  
El objetivo es medir tiempos de envío, cómputo y recolección y analizar cómo escala el k-NN usando `mpi4py`. El rank 0 opcionalmente proyecta el dataset con PCA para visualizar etiquetas reales vs predicciones.


**Autores:**  

Nieto Espinoza, Brajan E.  
[brajan.nieto@utec.edu.pe](mailto:brajan.nieto@utec.edu.pe)

Tapia Chasquibol,  Mateo J.  
[mateo,.tapia@utec.edu.pe](mailto:mateo.tapia@utec.edu.pe)

<img src="https://pregrado.utec.edu.pe/sites/default/files/logo-utec-h_0_0.svg" width="190" alt="Logo UTEC" loading="lazy" typeof="foaf:Image">      

---

### 0. Librerías

In [3]:
!pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.8 MB/s eta 0:00:00


In [4]:
from mpi4py import MPI
import numpy as np
from collections import Counter
import sys

### Imports Condicionales
Solo el rank 0 importa sklearn (generación de datos y PCA) y la parte de visualización (comentada).


In [5]:
if MPI.COMM_WORLD.Get_rank() == 0:
    from sklearn.datasets import make_classification
    from sklearn.decomposition import PCA


### Función de distancia
Devuelve un vector con las distancias euclidianas de cada fila de a hacia b.


In [ ]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2, axis=1))

### Configuración MPI



*   comm: comunicador global con todos los procesos.
*   rank: id del proceso actual (0 a size-1).
*   size: número total de procesos lanzados (por ej. -np 4 ⇒ 4).

In [ ]:
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### Parametros



Orden de argumentos:

  

*   argv[1] = train_size (opcional; default 1000)
*   argv[2] = test_size  (opcional; default 200)
*   argv[3] = k          (opcional; default 3)
*   argv[4] = num_features (opcional; default 10)

In [ ]:
train_size   = int(sys.argv[1]) if len(sys.argv) >= 2 else 1000
test_size    = int(sys.argv[2]) if len(sys.argv) >= 3 else 200
k            = int(sys.argv[3]) if len(sys.argv) >= 4 else 3
num_features = int(sys.argv[4]) if len(sys.argv) >= 5 else 10